# How to contribute?

Here we report a simple guide with the key features of RobustX in order to implement your own (robust) counterfactual explanation method.

# Setup preparation

In [2]:
# Import necessary components
from sklearn.model_selection import train_test_split
from robustx.lib.models.pytorch_models.SimpleNNModel import SimpleNNModel
from robustx.datasets.ExampleDatasets import get_example_dataset
from robustx.lib.tasks.ClassificationTask import ClassificationTask
from robustx.generators.robust_CE_methods.MCER import MCER
import numpy as np

# Load and preprocess dataset
dl = get_example_dataset("iris")
dl.preprocess(
    impute_strategy_numeric='mean',  # Impute missing numeric values with mean
    scale_method='minmax',           # Apply min-max scaling
    encode_categorical=False         # No categorical encoding needed (since no categorical features)
)

# remove the target column from the dataset that has labels 2
dl.data = dl.data[dl.data['target'] != 2]

# Load model, note some RecourseGenerators may only work with a certain type of model,
# e.g., MCE only works with a SimpleNNModel
model = SimpleNNModel(4, [10], 1, seed=0)

target_column = "target"
X_train, X_test, y_train, y_test = train_test_split(dl.data.drop(columns=[target_column]), dl.data[target_column], test_size=0.35, random_state=0)
model.train(X_train, y_train)

print("model accuracy: ", model.compute_accuracy(X_test.values, y_test.values))

# Create task
task = ClassificationTask(model, dl)

model accuracy:  0.5142857432365417


# Example of an already implemented CE generation method in RobustX

In [3]:
# Each counterfactual explanation generator takes the task on creation, it can also take a custom distance function, but for now we will use the default one.
ce_gen = MCER(task)

# Get negative instances, the default column_name is always "target" but you can set it to the name of your dataset's target variable
negs = dl.get_negative_instances(neg_value=0, column_name="target")
print("Negative instances shape: ", negs.shape)
print(f"Example of a prediction for a negative instance:\n")
print(negs.head(1))
print("Output: ", model.predict(negs.head(1)).values.item())
print("Class: ", int(model.predict(negs.head(1)).values.item() > 0.5))  # Assuming binary classification with threshold 0.5

# You can generate for a set of instances stored in a DataFrame
print("\nGenerating counterfactual explanations using STCE for the first 5 negative instances:")
ce = ce_gen.generate_for_instance(negs.iloc[0], delta=0.0, bias_delta=0.0)
print(ce)
print("Output: ", model.predict(ce).values.item())
print("Class: ", int(model.predict(ce).values.item() > 0.5))  # Assuming binary classification with threshold 0.5

Restricted license - for non-production use only - expires 2025-11-24
Negative instances shape:  (50, 4)
Example of a prediction for a negative instance:

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0           0.222222             0.625           0.067797          0.041667
Output:  0.46254587173461914
Class:  0

Generating counterfactual explanations using STCE for the first 5 negative instances:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0            0.40527          0.227558           0.067797          0.041667
Output:  0.5000250339508057
Class:  1


In [4]:
# You can also implement a method to generate CEs for all the negative instance in one shot
ces = ce_gen.generate_for_all(neg_value=0, column_name="target")
print("All outputs are positive? ", np.all(model.predict(ces)>0.5))

All outputs are positive?  True


# Implementing your own CE Generator

Here is an example of creating your own RecourseGenerator. Let's make a simple one which gets
n different positive instances and chooses a random one. Let's say it also allows a random seed value.

In [5]:
from robustx.generators.CEGenerator import CEGenerator
import pandas as pd

# Implement the RecourseGenerator class
class RandomCE(CEGenerator):

    # You must implement the _generation_method function, this returns the CE for a given
    # instance, if you take any extra arguments make sure to specify them before **kwargs,
    # like we have done for n and seed (they must have some default value)
    def _generation_method(self, instance, column_name="target", neg_value=0, n=50, seed=None, **kwargs):
        # Remember, the RecourseGenerator has access to its Task! Use this to get access to your dataset or model,
        # or to use any of their methods, here we use the ClassificationTask's get_random_positive_instance() method
        pos = pd.concat([self.task.get_random_positive_instance(neg_value=neg_value, column_name=column_name) for _ in range(n)])

        # Depending on whether a seed is provided, we return a random positive - the result must be a DataFrame
        if seed is None:
            return pos.sample(n=1)

        return pos.sample(n=1, random_state=seed)

Within the CEGenerator you can access:

- The Task - self.Task
- The DatasetLoader - self.task.training_data
- The BaseModel - self.task.model

and their respective methods. If your method needs additional arguments, you can put them in the function signature
but do NOT remove any other arguments (including **kwargs). Remember to return a DataFrame!

Here is our new CE in use below:

In [6]:
# Create RecourseGenerator
random_ce = RandomCE(task)

# Test it
ces = random_ce.generate_for_all()
print(ces)

    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0            0.694444          0.333333           0.644068          0.541667
1            0.472222          0.083333           0.508475          0.375000
2            0.694444          0.333333           0.644068          0.541667
3            0.472222          0.083333           0.508475          0.375000
4            0.527778          0.083333           0.593220          0.583333
..                ...               ...                ...               ...
91           0.694444          0.333333           0.644068          0.541667
92           0.555556          0.125000           0.576271          0.500000
93           0.694444          0.333333           0.644068          0.541667
94           0.472222          0.083333           0.508475          0.375000
95           0.555556          0.125000           0.576271          0.500000

[96 rows x 4 columns]


We can verify it by seeing all the predictions for the CEs are positive.

In [7]:
print(model.predict(ces))

           0
0   0.501608
1   0.505629
2   0.501608
3   0.505629
4   0.502795
..       ...
91  0.501608
92  0.505989
93  0.501608
94  0.505629
95  0.505989

[96 rows x 1 columns]


# Benchmarking your method

After you have finished implementing your method, you can include it into DefaultBenchmark.py file and test it against other methods supported in the library using this lines of code:

In [ ]:
from robustx.lib.DefaultBenchmark import default_benchmark
methods = ["KDTreeNNCE", "MCER"]
evaluations = ["Validity", "Distance"]
default_benchmark(task, methods, evaluations, neg_value=0, column_name="target", delta=0.005)

+------------+----------------------+------------+------------+
| Method     |   Execution Time (s) |   Validity |   Distance |
+============+======================+============+============+
| KDTreeNNCE |            0.0493832 |          1 |   0.533264 |
+------------+----------------------+------------+------------+
| MCER       |            0.938715  |          1 |   0.407967 |
+------------+----------------------+------------+------------+
